# Code Notebook Draft as of 06/26/2021

In [ ]:
# Import the necessary libraries
import pandas as pd

In [ ]:
# Load all raw datasets first: 
# c/o BONG
raw_harvard_election_returns_main = pd.read_csv('./data/HarvardData_countypres_2000-2020.csv')
raw_nytimes_dataset = pd.read_csv('./data/NYTimes_covidcasedata.csv')
raw_facebook_social_connectedness_index = pd.read_csv('./data/Facebook_Cornell_ERS_SocialConnectednessIndex.csv')
raw_landareas_counties = pd.read_csv('./data/AgriCensus_LandAreas.csv')
# c/o BONG
raw_bls_unemployment_2014 = pd.read_csv('./data/BLS_LaborUnemployment_2014_csv.csv')
raw_bls_unemployment_2015 = pd.read_csv('./data/BLS_LaborUnemployment_2015_csv.csv')
raw_bls_unemployment_2016 = pd.read_csv('./data/BLS_LaborUnemployment_2016_csv.csv')
raw_ers_detailed_labor = pd.read_csv('./data/ERS_RurAtlas_DetailedLabor.csv')
raw_ers_migration = pd.read_csv('./data/ERS_RurAtlas_Migration.csv')
raw_ers_urban_rural = pd.read_csv('./data/ERS_RurAtlas_UrbanRuralClassification.csv')
raw_fcc_health_and_broadband = pd.read_csv('./data/FCC_Health_and_Broadband.csv')
raw_harvard_streets_and_intersections = pd.read_csv('./data/HarvardData_streets_intersections.csv')
raw_noaa_storm_events = pd.read_csv('./data/NOAA_StormEvent_Fatalities_2016.csv')
raw_openicpsr_crime = pd.read_csv('./data/OPENICPSR_Crime.csv')
raw_pennstateagri_social_capital_variables = pd.read_csv('./data/PennState_AgriScienceDept_Religion_and_Social_Capital.csv')

/shared-libs/python3.9/py-core/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (2,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,36,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,267,269,303,304) have mixed types.Specify dtype option on

In [ ]:
# Begin investigating the raw datasets for dimension reduction
raw_nytimes_dataset.shape

(1439876, 6)

In [ ]:
raw_harvard_election_returns_main.shape
raw_harvard_election_returns_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72617 entries, 0 to 72616
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   year            72617 non-null  int64  
 1   state           72617 non-null  object 
 2   state_po        72553 non-null  object 
 3   county_name     72617 non-null  object 
 4   county_fips     72560 non-null  float64
 5   office          72617 non-null  object 
 6   candidate       72617 non-null  object 
 7   party           72617 non-null  object 
 8   candidatevotes  72212 non-null  float64
 9   totalvotes      72612 non-null  float64
 10  version         72617 non-null  int64  
 11  mode            72617 non-null  object 
dtypes: float64(3), int64(2), object(7)
memory usage: 6.6+ MB


In [ ]:
raw_facebook_social_connectedness_index.columns
raw_facebook_social_connectedness_index.info()
# Social Index Density within 50 mile, 100 mile and 500 mile radius:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3106 entries, 0 to 3105
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   FIPS          3106 non-null   int64  
 1   sh050m        3106 non-null   float64
 2   sh100m        3106 non-null   float64
 3   sh500m        3106 non-null   float64
 4   income        3106 non-null   float64
 5   migration     3106 non-null   float64
 6   birth         3106 non-null   float64
 7   death         3106 non-null   float64
 8   education     3106 non-null   float64
 9   unemployment  3106 non-null   float64
 10  election      3106 non-null   float64
dtypes: float64(10), int64(1)
memory usage: 267.0 KB


In [ ]:
raw_landareas_counties.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3234 entries, 0 to 3233
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   STATE FIPS                 3234 non-null   int64 
 1   COUNTY FIPS                3234 non-null   int64 
 2   STATE/TERRITORY NAME       3234 non-null   object
 3   COUNTY NAME                3234 non-null   object
 4    2010 POPULATION           3234 non-null   object
 5    2015 POPULATION           3218 non-null   object
 6    LAND AREA (Sqare Miles)   3234 non-null   object
dtypes: int64(2), object(5)
memory usage: 177.0+ KB


In [ ]:
raw_bls_unemployment_2014.head(5)

,LAUS_Code,S_Code,C_Code,County,Year,Unnamed: 5,Labor_Force,Employed,Unemployed,Unemp_Rate
0,CN0100100000000,1.0,1.0,"Autauga County, AL",2014.0,NaN,"25,639","24,150","1,489",5.8
1,CN0100300000000,1.0,3.0,"Baldwin County, AL",2014.0,NaN,"86,546","81,265","5,281",6.1
2,CN0100500000000,1.0,5.0,"Barbour County, AL",2014.0,NaN,"8,859","7,930",929,10.5
3,CN0100700000000,1.0,7.0,"Bibb County, AL",2014.0,NaN,"8,573","7,959",614,7.2
4,CN0100900000000,1.0,9.0,"Blount County, AL",2014.0,NaN,"24,571","23,074","1,497",6.1


In [ ]:
raw_ers_detailed_labor.head(2)

,FIPS,State,County,UnempRate2020,PctEmpChange1920,UnempRate2019,PctEmpChange1819,UnempRate2018,UnempRate2017,UnempRate2016,...,NumUnemployed2015,NumUnemployed2014,NumEmployed2014,NumCivLaborforce2014,NumUnemployed2013,NumCivLaborforce2013,NumUnemployed2007,NumEmployed2007,NumCivLaborforce2007,NumCivLaborforce2017
0,0,US,United States,8.1,-6.0,3.7,1.3,3.9,4.4,4.9,...,8285731.0,9617207.0,146318952.0,155936159.0,11457241.0,155362278.0,7034917.0,145156133.0,152191050.0,160214378.0
1,1000,AL,Alabama,5.9,-3.2,3.0,2.3,3.9,4.6,5.9,...,131852.0,146010.0,2018705.0,2164715.0,159274.0,2172102.0,88418.0,2092030.0,2180448.0,2172455.0


In [ ]:
raw_ers_migration.head(2)

,FIPS,State,County,PopChangeRate1819,PopChangeRate1019,TotalPopEst2019,NetMigrationRate1019,NaturalChangeRate1019,Net_International_Migration_Rate_2010_2019,PopChangeRate0010,...,TotalPopEst2014,TotalPopEst2011,Net_International_Migration_2010_2019,NaturalChange1019,TotalPopEst2015,TotalPopEst2016,TotalPopEst2017,NetMigration1019,TotalPopEst2018,TotalPopEstBase2010
0,0,US,United States,0.475,6.116,328239523,2.485,3.631,2.485,9.71,...,318301008,311556874,7685444.0,11232413.0,320635163,322941311,324985539,7685444.0,326687501,308758105
1,1000,AL,Alabama,0.317,2.461,4903185,1.059,1.402,0.809,7.48,...,4841799,4799069,38731.0,67083.0,4852347,4863525,4874486,50665.0,4887681,4780125


In [ ]:
raw_ers_urban_rural.head(2)

,FIPStxt,State,County,RuralUrbanContinuumCode2013,UrbanInfluenceCode2013,RuralUrbanContinuumCode2003,UrbanInfluenceCode2003,Metro2013,Nonmetro2013,Micropolitan2013,...,FarmDependent2003,ManufacturingDependent2000,LowEducation2000,RetirementDestination2000,PersistentPoverty2000,Noncore2013,Type_2015_Nonspecialized_NO,Metro_Adjacent2013,PersistentChildPoverty2004,RecreationDependent2000
0,1001,AL,Autauga,2.0,2.0,2.0,2.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1003,AL,Baldwin,3.0,2.0,4.0,5.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
raw_fcc_health_and_broadband.head(3)

,FID,geography_type,geography_id,geography_desc,submission_cycle,geom,centroid,res_concxns_pct,pctpopwbbacc,pctpopwobbacc,...,Unnamed: 343,Unnamed: 344,Unnamed: 345,Unnamed: 346,Unnamed: 347,Unnamed: 348,Unnamed: 349,Unnamed: 350,Unnamed: 351,Unnamed: 352
0,c2hgis_county.fid--3df29475_17a3fe20314_5ad2,county,21109,Jackson County,2015-09,"MULTIPOLYGON (((-84.205306 37.503062, -84.1994...",POINT (-84.00617164501776 37.41979135091643),3,99.6,0.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,c2hgis_county.fid--3df29475_17a3fe20314_5ad3,county,37005,Alleghany County,2015-09,MULTIPOLYGON (((-81.3531692825529 36.574724003...,POINT (-81.127710400513 36.49135989086295),4,87.7,12.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,c2hgis_county.fid--3df29475_17a3fe20314_5ad4,county,13123,Gilmer County,2015-09,"MULTIPOLYGON (((-84.65709799999999 34.728905, ...",POINT (-84.45574304407471 34.69141719651767),5,95.1,4.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
raw_harvard_streets_and_intersections.head(2)

,county,geoid,area_km,avg_neighbor_degree_avg,avg_weighted_neighbor_degree_avg,circuity_avg,cluster_coeff_avg,cluster_coeff_weighted_avg,degree_centrality_avg,edge_density_km,...,node_count,node_density_km,pagerank_max,pagerank_min,self_loop_proportion,street_density_km,street_length_avg,street_length_total,street_segments_count,streets_per_node_avg
0,Autauga County,1001,1539.614693,2.801244,0.021544,1.095906,0.045214,0.000854,0.001109,2519.774067,...,4315,2.802649,0.000770,0.000035,0.010265,1282.794436,371.521663,1975009.162,5316,2.455156
1,Baldwin County,1003,4117.605847,2.753929,0.026284,1.078592,0.037953,0.000241,0.000254,2772.858997,...,18730,4.548760,0.000203,0.000008,0.015718,1465.780821,254.319386,6035507.677,23732,2.501281


In [ ]:
raw_noaa_storm_events.head(2)

,FAT_YEARMONTH,FAT_DAY,FAT_TIME,FATALITY_ID,EVENT_ID,FATALITY_TYPE,FATALITY_DATE,FATALITY_AGE,FATALITY_SEX,FATALITY_LOCATION,EVENT_YEARMONTH
0,201602,24,0,30075,619440,D,02/24/2016 00:00:00,78.0,M,Outside/Open Areas,201602.0
1,201603,9,0,30181,626789,D,03/09/2016 00:00:00,56.0,M,Outside/Open Areas,201603.0


In [ ]:
raw_openicpsr_crime.head(2)

,STCOFIPS,FIPS_ST,FIPS_CTY,YEAR,STUDYNO,EDITION,PART,IDNO,CPOPARST,CPOPCRIM,...,VIOL,PROPERTY,MURDER,RAPE,ROBBERY,AGASSLT,BURGLRY,LARCENY,MVTHEFT,ARSON
0,1001,1,1,2002,ICPSR Study No,"1st ed, August 2004",Crimes-County,1,44959,24424,...,NaN,NaN,0,18,35,48,172,1023,48,13
1,1001,1,1,2003,4360,"1st ed, August 2004",Crimes-County,1,46808,45749,...,NaN,NaN,3,29,34,95,326,1235,107,3


In [ ]:
raw_pennstateagri_social_capital_variables.head(3)

,FIPS,County_Name,Religious2014,Civic2014,Business2014,Political2014,Professional2014,Labor2014,Bowling2014,Recreational2014,...,nccs2014,sk2014,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,1001.0,"Autauga County, AL",53.0,7.0,3.0,0.0,1.0,4.0,1.0,5.0,...,157.0,-0.631003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1003.0,"Baldwin County, AL",169.0,16.0,9.0,0.0,3.0,1.0,1.0,25.0,...,718.0,-0.555396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1005.0,"Barbour County, AL",19.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,92.0,-0.891036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## DATA SOURCE AND DESCRIPTION
Most of the datasets from our multiple sources are descriptive of counties in the United States. One county is one record, in general. 

### WHAT IS THE TARGET VARIABLE?
The target variable is the classification of political leaning.

### WHAT ARE THE PREDICTOR VARIABLES UNDER CONSIDERATION?
The predictor variables are divided into three groups:
#### Cultural
- Urban and Rural Spectrum Classification
- Facebook Social Connectedness Index Per County
- Social Capital Variables from Penn State (including a Religion variable)

#### Physical
- Land Area (for scaling mainly)
- Health Variables
- Broadband
- Streets and Intersections
- Storm Fatalities for 2016

#### Socioeconomic
- Labor and Unemployment
- Migration
- Crime

## DATA CLEANING, AGGREGATIONS, AND TRANSFORMATION

## DATA VISUALIZATIONS AND EDA

## DIMENSION REDUCTION AND DETERMINING PREDICTORS

## DATA MODELS

## MODEL EVALUATIONS

## MODEL SELECTION AND FINDINGS

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5e29d49d-e8a5-4b79-a7a8-5fff20f26b9d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>